# Reminder to startup the Unity render binary if it is not already running!

In [1]:
import sys
import numpy as np
from IPython.display import HTML, display

class RRT_Node:
    def __init__(self, pos):
        self.pos = pos
        self.edges = np.array([])

class RRT_Graph:
    def __init__(self, nodes=np.array([])):
        self.nodes = nodes
    def add_node(self, node):
        self.nodes = np.append(self.nodes, node)
    def add_edge(self, node1, node2):
        node1.edges = np.append(node1.edges, node2)
        node2.edges = np.append(node2.edges, node1)
        
    def nearest_node(self, v):
        dist = None
        nearest = None
        for node in self.nodes:
            node_dist = np.linalg.norm(v - node.pos)
            if dist is None or node_dist < dist:
                dist = node_dist
                nearest = node
        return nearest
    
    # BFS
    def path_exists(self, source, sink):
        if source == sink:
            return True
        visited = {}
        for n in self.nodes:
            visited[n] = False
        queue = [source]
        visited[source] = True
        while queue:
            top = queue.pop(0)
            for connected in top.edges:
                if connected == sink:
                    return True
                if not visited[connected]:
                    visited[connected] = True
                    queue = np.append(queue, connected)
        return False

    # Dijkstra
    def shortest_path(self, source, sink):
        if source == sink:
            return True
        visited = {}
        dist = {}
        parent = {}
        for n in self.nodes:
            visited[n] = False
            parent[n] = None
            dist[n] = [sys.maxsize]
        
        dist[source] = 0
        queue = [source]
        while queue:
            curr = queue.pop(0)
            for n in curr.edges:
                if not visited[n]:
                    queue = np.append(queue, n)
                    new_dist = dist[curr] + np.linalg.norm(curr.pos - n.pos)
                    if dist[n] > new_dist:
                        parent[n] = curr
                        dist[n] = new_dist
            visited[curr] = True
            
        # helper, assumes the parent exists
        def get_path(parent, curr, path):
            path = get_path(parent, parent[curr], np.append(path, curr))
            return path
        
        if not parent[sink]:
            return None # no valid path
        else:
            return get_path(parent, sink, np.array([]))

In [2]:
#!/usr/bin/env python
# coding: utf-8
%matplotlib inline
%reload_ext autoreload
%autoreload 2
sys.path.insert(0, '../')
from flightgoggles.env import *

def gen_rand_pos(bound1, bound2):
    rand_pos = bound1 + (bound2 - bound1) * [np.random.random_sample(),
                                             np.random.random_sample(),
                                             np.random.random_sample()]
    return rand_pos

def willCollide(env, start_pose, end_pose):
    EPSILON = 0.03
    env.set_state_vehicle(vehicle_id="uav1",
                          position=start_pose[:3],
                          attitude_euler_angle=np.array([0., 0., start_pose[3]]))
    # proceed_waypoint -> fix a location to go toward; duration is the delta time in PID loop
    curr_pos = start_pose[:3]
    while np.linalg.norm(curr_pos - end_pose[:3]) > EPSILON:
        collided = env.proceed_waypoint(vehicle_id="uav1", waypoint_command=end_pose, duration=0.01)
        curr_pos = env.get_state("uav1")["position"]
        if collided:
            return True
    return False

if __name__ == "__main__":
    env = flightgoggles_env(
        cfg_dir="../config",
        cfg_fgclient="FlightGogglesClient_testing.yaml",
        cfg_uav="multicopterDynamicsSim.yaml")
    start_pos = np.array([0, -15., -2.])
    start_att = np.array([0.,0.,-np.pi/2])
    
    target_pos = start_pos + np.array([3, 0., 0.]) #np.array([9., -15., -2.])
    target_att = start_att + np.array([0., 0., np.pi/2]) #np.array([0., 0., 0.])
    target_pose = np.append(target_pos, target_att[2])
    
    start_node = RRT_Node(start_pos)
    target_node = RRT_Node(target_pos)
    graph = RRT_Graph(np.array([start_node, target_node]))
    
    trials = 100
    for _ in range(trials): # if after trials number of tries, assume the target_pose is not reachable from current pose
        rand_pos = gen_rand_pos(start_pos, target_pos)
        print(rand_pos)
        nearest_node = graph.nearest_node(rand_pos)

        # ignore delta steps for now
        if not willCollide(env, np.append(nearest_node.pos, 0.),
                                np.append(rand_pos, 0.)):
            new_node = RRT_Node(rand_pos)
            graph.add_node(new_node)
            graph.add_edge(nearest_node, new_node)
    
    
    
    # -> get the path  dijkstra or something
    # TODO!!!!! FIll out the graph function
    path = graph.shortest_path(start_node, target_node)
    print(path)
    
    
    # follow the path with uav1, see cell below
    env.close()












[  2.07287064 -15.          -2.        ]
[  0.63425919 -15.          -2.        ]
[  1.12476866 -15.          -2.        ]
[  0.78187072 -15.          -2.        ]
[  2.2314255 -15.         -2.       ]
[  1.84937633 -15.          -2.        ]
[  2.23053089 -15.          -2.        ]
[  2.3374169 -15.         -2.       ]
[  1.34274516 -15.          -2.        ]
[  1.69526509 -15.          -2.        ]
[  0.84389004 -15.          -2.        ]
[  0.52809006 -15.          -2.        ]
[  0.24374216 -15.          -2.        ]
[  1.28494686 -15.          -2.        ]
[  1.01030739 -15.          -2.        ]
[  1.91630973 -15.          -2.        ]
[  1.19377329 -15.          -2.        ]
[  0.57874708 -15.          -2.        ]
[  0.03034059 -15.          -2.        ]
[  0.67765773 -15.          -2.        ]
[  2.08637522 -15.          -2.        ]
[  1.39314846 -15.          -2.        ]
[  1.1605882 -15.         -2.       ]
[  2.29656063 -15.          -2.        ]
[  2.60241102 

AttributeError: 'RRT_Node' object has no attribute 'id'

In [ ]:
# -> get the path  dijkstra or something
    # TODO!!!!! FIll out the graph function
    path = graph.shortest_path(start_pos, target_pos)
    
    
    
    env.set_state_vehicle(vehicle_id="uav1",
                          position=start_pos,
                          attitude_euler_angle=start_att)

    # position and attitude -> pose
    print("Start pos: {}".format(start_pos))
    print("Start attitude: {}".format(start_att))
    print("Target pose: {}".format(target_pose))
        
    # proceed_waypoint -> fix a location to go toward; duration is the delta time in PID loop
    #for j in range(600):
    curr_pos = start_pos
    while np.linalg.norm(curr_pos - target_pos) > 0.03:
        collided = env.proceed_waypoint(vehicle_id="uav1", waypoint_command=target_pose, duration=0.01)
        curr_pos = env.get_state("uav1")["position"]
        if collided:
            break
    
    env.plot_state(vehicle_id="uav1", attribute="position")
#     env.save_logs(vehicle_id="uav1", save_dir="data/")
    img = env.get_camera_image(camera_id="cam1")
   # print(img[-1]["timestamp"])
    
    final_pos = env.get_state("uav1")["position"]
    final_att = env.get_state("uav1")["attitude_euler_angle"]
#     print("Final pos: {}".format(final_pos))
#     print("Final attitude: {}".format(final_att))
    with np.printoptions(precision=2, suppress=True):
        print("Final pose", np.append(final_pos, final_att[2]))
    
    # env.plot_state_camera(camera_id=None) last cmaera img. only 1 camera on uav1

    ani_set = env.plot_state_video(flag_save=False, filename="uav", dpi=400)
    if "cam1" in ani_set.keys():
        display(HTML(ani_set["cam1"].to_html5_video()))
    env.close()